In [2]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv("../../91APPdataset/Orders.csv", header = 0, sep = ',')

/home/kevin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
dict_region = {"北部":['臺北', '台北', '新北', '基隆', '宜蘭', '桃園', '新竹'],
               "中部":['苗栗', '臺中', '台中', '彰化', '南投', '雲林'],
               "南部":['嘉義', '臺南', '台南', '高雄', '屏東', '澎湖'],
               "東部":['花蓮', '台東', '臺東'],
               "離島":['金門', '馬祖', '澎湖', '綠島', '蘭嶼']
              }

In [4]:
df['new_DateId'] = pd.to_datetime(df['DateId'].astype(str), format='%Y%m%d')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5495276 entries, 0 to 5495275
Data columns (total 29 columns):
DateId                         int64
MemberId                       int64
OrderGroupCode                 object
TrackSourceTypeDef             object
TrackDeviceTypeDef             object
PayProfileTypeDef              object
SalesOrderSlaveId              int64
SalePageId                     int64
IsMajor                        bool
IsGift                         bool
IsSalePageGift                 bool
Quantity                       int64
UnitPrice                      float64
PromotionDiscount              float64
ECouponId                      int64
ECouponDiscount                float64
SalesOrderSlaveTotalPayment    float64
SalesOrderSlaveDateTime        object
SalesOrderReceiverId           int64
City                           object
District                       object
ZipCode                        float64
StoreName                      object
StoreId              

* 依照city做分類

In [6]:
data = df.groupby(by='City')['Quantity'].sum()

In [7]:
data

City
1st         45
25          10
<fo         41
Aya          2
Cha          1
Daf          2
Hsi          3
Hy           6
Ins          2
Jia          3
Kao          5
Keb          3
Kot          2
Nan         28
New         32
No.          3
Quậ          6
Tai          8
Tao         18
Xin          5
Đại          1
南投縣      74939
台中市     815918
台北市     705464
台南市     407470
台東市        431
台東縣      34786
嘉義市      55276
嘉義縣      68262
基隆市      89367
宜蘭縣      87111
屏東市        776
屏東縣     125121
彰化縣     201898
新北市    1196609
新竹市     148414
新竹縣     129924
桃園市     599460
桃園縣         93
澎湖縣      20190
花蓮縣      69475
苗栗縣     108647
雲林縣      88407
高雄市     580556
Name: Quantity, dtype: int64

## 發現亂碼、NaN，以及新舊制的問題

### 1. 解決亂碼、NaN，判斷是否為漢字

In [9]:
# 判斷一個unicode是否為漢字
def is_chinese(uchar):
    if '\u4e00' <= uchar <= '\u9fff':
        return True
    else:
        return False

In [12]:
print(is_chinese("New"))
print(is_chinese('我x'))

False
True


In [13]:
array_clean1 = np.array(df)
ls_city = list(df['City'])
type(ls_city)

list

In [15]:
drop_list = list() #找出需要清的index
for i in range(len(ls_city)):
    try:
        if not (is_chinese(ls_city[i])):
            # print(str(i)+':'+(ls_city[i])'+' will be dropped')
            drop_list.append(i)
    except: 
        # print('error at' + str(df['City'][i]+', i = '+str(i))
        drop_list.append(i)
        continue

In [18]:
array_clean1 = np.delete(array_clean1, drop_list, 0) #依 drop_list 刪除列

In [19]:
df_clean1 = pd.DataFrame(array_clean1, columns = list(df)) # list(df)可得到df的header

In [21]:
df_clean1.groupby(by='City')['Quantity'].sum()

City
南投縣      74939
台中市     815918
台北市     705464
台南市     407470
台東市        431
台東縣      34786
嘉義市      55276
嘉義縣      68262
基隆市      89367
宜蘭縣      87111
屏東市        776
屏東縣     125121
彰化縣     201898
新北市    1196609
新竹市     148414
新竹縣     129924
桃園市     599460
桃園縣         93
澎湖縣      20190
花蓮縣      69475
苗栗縣     108647
雲林縣      88407
高雄市     580556
Name: Quantity, dtype: int64

In [24]:
df_clean1.to_pickle('df_clean1.pkl')

## 2. 解決新舊制問題

In [ ]:
df_clean2 = pd.read_pickle('df_clean1.pkl')